In [ ]:
!pip install flask opencv-python numpy easyocr pyngrok python-dotenv
!sudo apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.


In [ ]:
import cv2
import numpy as np
import easyocr
from datetime import datetime
from flask import Flask, render_template, Response
import sqlite3
import os
from threading import Thread
from pyngrok import ngrok, conf
from IPython.display import HTML, display

# Initialize Flask
app = Flask(__name__)

# Initialize EasyOCR
reader = easyocr.Reader(['en'], gpu=True)

# Database setup
def init_db():
    conn = sqlite3.connect('violations.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS violations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            vehicle_number TEXT,
            violation_type TEXT,
            speed FLOAT,
            timestamp DATETIME,
            screenshot_path TEXT
        )
    ''')
    conn.commit()
    conn.close()

# Video processing
video_path = 'traffic.mp4'  # Upload your video to Colab first
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
violations = []
processing = True

def detect_license_plate(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    result = reader.readtext(gray)
    return result[0][1] if result else "UNKNOWN"

def estimate_speed(prev_pos, curr_pos):
    distance_pixels = np.linalg.norm(np.array(curr_pos) - np.array(prev_pos))
    return distance_pixels * 0.1 * fps * 3.6  # Convert to km/h

def process_video():
    global violations
    prev_positions = {}

    while processing:
        ret, frame = cap.read()
        if not ret:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            continue

        # Vehicle detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)
        contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            if cv2.contourArea(contour) > 1000:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                # Track vehicle
                vehicle_id = f"{x}_{y}"
                if vehicle_id in prev_positions:
                    speed = estimate_speed(prev_positions[vehicle_id], (x, y))

                    if speed > 50:  # Speeding threshold
                        license_plate = detect_license_plate(frame[y:y+h, x:x+w])
                        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        screenshot_path = f"static/violation_{len(violations)}.jpg"

                        # Save violation
                        conn = sqlite3.connect('violations.db')
                        cursor = conn.cursor()
                        cursor.execute('''
                            INSERT INTO violations VALUES (NULL, ?, ?, ?, ?, ?)
                        ''', (license_plate, 'SPEEDING', speed, timestamp, screenshot_path))
                        conn.commit()
                        conn.close()

                        # Save screenshot
                        cv2.imwrite(screenshot_path, frame[y:y+h, x:x+w])
                        violations.append({
                            'vehicle_number': license_plate,
                            'violation_type': 'SPEEDING',
                            'speed': speed,
                            'timestamp': timestamp,
                            'screenshot': screenshot_path
                        })

                prev_positions[vehicle_id] = (x, y)

# Flask routes
@app.route('/')
def live_feed():
    return render_template('live.html')

@app.route('/violations')
def violation_log():
    conn = sqlite3.connect('violations.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM violations ORDER BY timestamp DESC')
    violations = cursor.fetchall()
    conn.close()
    return render_template('violations.html', violations=violations)

def generate_frames():
    while True:
        ret, frame = cap.read()
        if not ret:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            continue

        # Draw processing info
        cv2.putText(frame, f"Detected Violations: {len(violations)}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(),
                   mimetype='multipart/x-mixed-replace; boundary=frame')

# Create necessary directories
os.makedirs('templates', exist_ok=True)
os.makedirs('static', exist_ok=True)

# Create HTML templates
with open('templates/live.html', 'w') as f:
    f.write('''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Live Traffic Monitoring</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; }
            .container { max-width: 1000px; margin: 0 auto; }
            .video-container { margin: 20px 0; border: 2px solid #333; }
            .nav-button {
                display: inline-block;
                padding: 10px 20px;
                background: #4CAF50;
                color: white;
                text-decoration: none;
                border-radius: 5px;
                margin-right: 10px;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Live Traffic Violation Detection</h1>
            <div class="video-container">
                <img src="{{ url_for('video_feed') }}" width="100%">
            </div>
            <div>
                <a href="{{ url_for('violation_log') }}" class="nav-button">View Violation Log</a>
                <a href="{{ url_for('live_feed') }}" class="nav-button">Refresh Live Feed</a>
            </div>
        </div>
    </body>
    </html>
    ''')

with open('templates/violations.html', 'w') as f:
    f.write('''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Violation Log</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; }
            .container { max-width: 1200px; margin: 0 auto; }
            table { width: 100%; border-collapse: collapse; margin-top: 20px; }
            th, td { padding: 12px; text-align: left; border-bottom: 1px solid #ddd; }
            th { background-color: #f2f2f2; }
            img.evidence { max-width: 200px; border: 1px solid #ddd; }
            .nav-button {
                display: inline-block;
                padding: 10px 20px;
                background: #4CAF50;
                color: white;
                text-decoration: none;
                border-radius: 5px;
                margin-bottom: 20px;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Traffic Violation Log</h1>
            <a href="{{ url_for('live_feed') }}" class="nav-button">Back to Live Feed</a>
            <table>
                <tr>
                    <th>License Plate</th>
                    <th>Violation</th>
                    <th>Speed</th>
                    <th>Timestamp</th>
                    <th>Evidence</th>
                </tr>
                {% for violation in violations %}
                <tr>
                    <td>{{ violation[1] }}</td>
                    <td>{{ violation[2] }}</td>
                    <td>{{ "%.1f"|format(violation[3]) }} km/h</td>
                    <td>{{ violation[4] }}</td>
                    <td><img class="evidence" src="{{ url_for('static', filename=violation[5].split('/')[-1]) }}"></td>
                </tr>
                {% endfor %}
            </table>
        </div>
    </body>
    </html>
    ''')

# Initialize database
init_db()

# Start video processing thread
processing_thread = Thread(target=process_video)
processing_thread.daemon = True
processing_thread.start()

# Try Ngrok connection (optional)
try:
    # First install ngrok if needed
    !curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null
    !echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list
    !sudo apt update && sudo apt install ngrok

    # Connect with your token
    !ngrok authtoken (Your token here)  # Replace with your actual token
    public_url = ngrok.connect(5000).public_url
    print(f" * Public URL: {public_url}")
except Exception as e:
    print(f"Ngrok connection failed: {e}")
    public_url = None

# Display in Colab
if public_url:
    display(HTML(f"""
    <div style="font-family: Arial; margin: 20px">
        <h2>Traffic Violation Detection System</h2>
        <p><strong>Public Dashboard URL:</strong> <a href="{public_url}" target="_blank">{public_url}</a></p>
        <iframe src="{public_url}" width="100%" height="600px" style="border: 1px solid #ddd"></iframe>
    </div>
    """))
else:
    display(HTML("""
    <div style="font-family: Arial; margin: 20px">
        <h2>Traffic Violation Detection System</h2>
        <p style="color: orange">Using local Flask server (no Ngrok)</p>
        <p>Run this command in a new cell to access:</p>
        <pre>from google.colab.output import eval_js; print(eval_js("google.colab.kernel.proxyPort(5000)"))</pre>
    </div>
    """))

# Run Flask app
app.run(host='0.0.0.0', port=5000)